## КРАТКОЕ ОПИСАНИЕ И ЦЕЛИ

Китов и дельфинов в этом наборе данных можно идентифицировать по формам, особенностям и отметинам (некоторые естественные, некоторые приобретенные) спинных плавников, спин, голов и боков. Некоторые виды и некоторые особи имеют очень отчетливые черты, другие гораздо менее отчетливы. Кроме того, индивидуальные особенности могут меняться с течением времени. В этом  датасете содержатся изображения более 15 000 уникальных особей морских млекопитающих из 30 различных видов, собранные из 28 различных исследовательских организаций. 

Отдельные особи были идентифицированы вручную и им присвоен индивидуальный идентификатор компанией marine researches, **задача** - правильно идентифицировать этих особей на изображениях. Это сложная задача, которая потенциально может привести к значительному прогрессу в понимании и защите морских млекопитающих по всему миру.


~ Ariadna, Sabina and Sonya

In [1]:
# библиотеки

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 # work with images
import matplotlib.pyplot as plt #visualization
import seaborn as sns #visualization
import os #path finder

from skimage import io
from skimage.color import rgb2gray
import plotly.express as px

In [2]:
# получение данных
path = '/kaggle/input/happy-whale-and-dolphin/'

sample_submission_path = "/kaggle/input/happy-whale-and-dolphin/sample_submission.csv"
train_images_path = "/kaggle/input/happy-whale-and-dolphin/train_images/"
train_path = "/kaggle/input/happy-whale-and-dolphin/train.csv"
test_images_path = "/kaggle/input/happy-whale-and-dolphin/test_images/"

os.listdir(path)

# Работа с данными

In [3]:
train_data = pd.read_csv(train_path)
train_data.head()

### Пример того, что должно получится в итоге

In [4]:
sample_submission = pd.read_csv(sample_submission_path)
sample_submission.head()


In [5]:
sample_submission.predictions[0]

#получается для каждого изображения нам нужно пять предсказаний

In [6]:
train_data.info()

### в тренировачном датасете 51033 входных данных

In [7]:
# посмотрим сколько уникальных видов

print(f'Всего : {train_data["species"].nunique()}')


### Наблюдения

1. при рассмотрении данных мы заметили пару орфографических ошибок, поэтому совершим объединение. 
2. также оказалось,что pilot_whale и globis оба являются short_finned_pilot_whale, поэтому они могут быть объединены 

*(информация взята из обссуждений, создатель соревнования подтвердил этот факт https://www.kaggle.com/c/happy-whale-and-dolphin/discussion/305468#1677103)

In [8]:
train_data['species'].replace({'bottlenose_dolpin':'bottlenose_dolphin',  #missing the 'h' in dolphin
                               'kiler_whale': 'killer_whale',             #missing the 'l' in killer
                               'globis':'short_finned_pilot_whale',       #correcting species names
                               'pilot_whale':'short_finned_pilot_whale'}, #correcting species names
                              inplace=True)

print(f'Тогда всего : {train_data["species"].nunique()}')

In [9]:
print('Количество особей каждого вида')
print(train_data['species'].value_counts())


print("Количество уникальных идентификаторов")
print(train_data['individual_id'].value_counts())

## Вывод

1. У некоторых видов гораздо больше изображений чем у других.
2. некоторые id были записаны много раз (до 400 изображений) в то время как другие были зарегистрированы только один раз

## Визуализируем число особей для каждого вида

In [10]:
fig = plt.figure(figsize=(15, 5))
sns.countplot(x=train_data['species'],
            order=train_data['species'].value_counts().index).set(title='Species Counts')
plt.xticks(rotation=90);

### Пока все...

### В планах создать новый столбец, в котором будет хранится информация о том, к какому семейству относится вид (дельфин или кит). Для этого необходимо провести небольшое исследования, чтобы правильно разделить виды на эти две группы